In [1]:
#import
import matplotlib.pyplot as plt
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, split, avg
from pyspark.sql.functions import split, trim, col, lower, regexp_replace, row_number, desc, expr, year
from pyspark.sql.window import Window
from pyspark.sql.functions import col, count, when

import time


In [2]:
# Nettoyer toute session existante
try:
    spark = SparkSession.builder.getOrCreate()
    spark.stop()
except Exception:
    pass

# Créer une nouvelle session Spark
spark = SparkSession.builder \
    .appName("TestSpark") \
    .master("local[*]") \
    .config("spark.local.dir", "/tmp/spark-temp") \
    .config("spark.executor.extraJavaOptions", "-Djava.security.manager=allow") \
    .config("spark.driver.extraJavaOptions", "-Djava.security.manager=allow") \
    .getOrCreate()

file_path_csv = "./data/en.openfoodfacts.org.products.csv"
file_path_parquet = "./data/en.openfoodfacts.org.products.parquet"


24/12/02 15:58:01 WARN Utils: Your hostname, Maxime.local resolves to a loopback address: 127.0.0.1; using 10.60.61.86 instead (on interface en0)
24/12/02 15:58:01 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/12/02 15:58:02 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
24/12/02 15:58:03 WARN SparkConf: Note that spark.local.dir will be overridden by the value set by the cluster manager (via SPARK_LOCAL_DIRS in mesos/standalone/kubernetes and LOCAL_DIRS in YARN).


In [3]:
try:
    start_time = time.time()
    # Reducing logs
    file_path_csv = "./data/en.openfoodfacts.org.products.csv"
    file_path_parquet = "./data/en.openfoodfacts.org.products.parquet"

    df = spark.read.format("csv") \
        .option("header", "true") \
        .option("delimiter", "\t") \
        .option("inferSchema", "true") \
        .option("encoding", "UTF-8") \
        .load(file_path_csv)

    # Save the DataFrame in Parquet format
    df.write.parquet(file_path_parquet, mode="overwrite")
    print("Données sauvegardées au format Parquet.")

    # Load the Parquet file for future analysis
    df_parquet = spark.read.parquet(file_path_parquet)
    print("Fichier Parquet chargé.")

# Check the schema to validate the conversion
#df_parquet.printSchema()
except Exception as e:
    print(e)

finally:
    end_time = time.time()
    elapsed_time = end_time - start_time
    print(f"Execution time: {elapsed_time:.2f} seconds")


24/12/02 15:58:36 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.
24/12/02 15:58:37 WARN MemoryManager: Total allocation exceeds 95,00% (1 020 054 720 bytes) of heap memory
Scaling row group sizes to 95,00% for 8 writers
24/12/02 15:58:37 WARN MemoryManager: Total allocation exceeds 95,00% (1 020 054 720 bytes) of heap memory
Scaling row group sizes to 84,44% for 9 writers
24/12/02 15:58:37 WARN MemoryManager: Total allocation exceeds 95,00% (1 020 054 720 bytes) of heap memory
Scaling row group sizes to 76,00% for 10 writers
24/12/02 15:58:37 WARN MemoryManager: Total allocation exceeds 95,00% (1 020 054 720 bytes) of heap memory
Scaling row group sizes to 69,09% for 11 writers
24/12/02 15:58:37 WARN MemoryManager: Total allocation exceeds 95,00% (1 020 054 720 bytes) of heap memory
Scaling row group sizes to 63,33% for 12 writers
24/12/02 15:58:51 WARN MemoryManager: Tota

Données sauvegardées au format Parquet.
Fichier Parquet chargé.
Execution time: 115.03 seconds


In [18]:
#read the parquet file
df_parquet = spark.read.parquet(file_path_parquet)
df_parquet.printSchema()


root
 |-- code: double (nullable = true)
 |-- url: string (nullable = true)
 |-- creator: string (nullable = true)
 |-- created_t: integer (nullable = true)
 |-- created_datetime: timestamp (nullable = true)
 |-- last_modified_t: integer (nullable = true)
 |-- last_modified_datetime: timestamp (nullable = true)
 |-- last_modified_by: string (nullable = true)
 |-- last_updated_t: integer (nullable = true)
 |-- last_updated_datetime: timestamp (nullable = true)
 |-- product_name: string (nullable = true)
 |-- abbreviated_product_name: string (nullable = true)
 |-- generic_name: string (nullable = true)
 |-- quantity: string (nullable = true)
 |-- packaging: string (nullable = true)
 |-- packaging_tags: string (nullable = true)
 |-- packaging_en: string (nullable = true)
 |-- packaging_text: string (nullable = true)
 |-- brands: string (nullable = true)
 |-- brands_tags: string (nullable = true)
 |-- categories: string (nullable = true)
 |-- categories_tags: string (nullable = true)
 |-- 

In [5]:
start_time = time.time()

# printSchema
df_parquet.printSchema()
print(" ")
df_parquet.show(20, truncate=False)
# Print number of columns
print(f"Number of columns: {len(df_parquet.columns)}")
print(" ")

# Print number of rows
print(f"Number of rows: {df_parquet.count()}")
print(" ")

# Calculate and display elapsed time
elapsed_time = time.time() - start_time
print("Loading completed.")
print(f"Total execution time: {elapsed_time:.2f} seconds")

root
 |-- code: double (nullable = true)
 |-- url: string (nullable = true)
 |-- creator: string (nullable = true)
 |-- created_t: integer (nullable = true)
 |-- created_datetime: timestamp (nullable = true)
 |-- last_modified_t: integer (nullable = true)
 |-- last_modified_datetime: timestamp (nullable = true)
 |-- last_modified_by: string (nullable = true)
 |-- last_updated_t: integer (nullable = true)
 |-- last_updated_datetime: timestamp (nullable = true)
 |-- product_name: string (nullable = true)
 |-- abbreviated_product_name: string (nullable = true)
 |-- generic_name: string (nullable = true)
 |-- quantity: string (nullable = true)
 |-- packaging: string (nullable = true)
 |-- packaging_tags: string (nullable = true)
 |-- packaging_en: string (nullable = true)
 |-- packaging_text: string (nullable = true)
 |-- brands: string (nullable = true)
 |-- brands_tags: string (nullable = true)
 |-- categories: string (nullable = true)
 |-- categories_tags: string (nullable = true)
 |-- 

#### Aperçu des données

In [8]:
start_time = time.time()

# Calculate missing data percentage for each column
total_rows = df_parquet.count()
missing_data = (
    df_parquet.select([
        (count(when(col(c).isNull() | (col(c) == ""), c)) / total_rows).alias(c)
        for c in df_parquet.columns
    ])
)

# Transform columns into rows (melt operation)
missing_data_melted = missing_data.selectExpr(
    "stack({0}, {1}) as (Column, MissingPercentage)".format(
        len(df_parquet.columns),
        ", ".join([f"'{col}', `{col}`" for col in df_parquet.columns])
    )
).filter(col("MissingPercentage").isNotNull()).orderBy(col("MissingPercentage").desc())

# Identify columns with 100% missing data
columns_to_drop = (
    missing_data_melted.filter(col("MissingPercentage") == 1.0)
    .select("Column")
    .rdd.flatMap(lambda x: x)
    .collect()
)

# Drop columns with 100% missing values
df_cleaned = df_parquet.drop(*columns_to_drop)

# Display the top 10 columns with the highest missing percentages
print("Top 10 columns with the highest missing percentages:")
missing_data_melted.show(10, truncate=False)

# Print dropped columns
print(f"Columns dropped due to 100% missing values: {columns_to_drop}")

end_time = time.time()
elapsed_time = end_time - start_time
print(f"Execution completed in {elapsed_time:.2f} seconds")

ConnectionRefusedError: [Errno 61] Connection refused

#### Nettouyer les données

In [6]:
start_time = time.time()

# Analyzing Duplicates in 'code', 'product_name', and 'brands'
duplicates = (
    df_parquet.groupBy("code", "product_name", "brands")
    .count()
    .filter(col("count") > 1)
)

# Affiche le nombre de doublons identifiés
print(f"There are {duplicates.count()} duplicate rows based on 'code', 'product_name', and 'brands'.")
duplicates.show(truncate=False)

# Remove duplicates where 'code', 'product_name', and 'brands' are the same
df_cleaned = df_parquet.dropDuplicates(["code", "product_name", "brands"])

print(f"Number of rows after removing duplicates: {df_cleaned.count()}")

end_time = time.time()
elapsed_time = end_time - start_time
print(f"Execution completed in {elapsed_time:.2f} seconds")


24/12/02 16:00:35 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
24/12/02 16:00:35 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
24/12/02 16:00:35 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
24/12/02 16:00:35 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
24/12/02 16:00:35 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
24/12/02 16:00:35 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.


There are 1968 duplicate rows based on 'code', 'product_name', and 'brands'.


24/12/02 16:00:40 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
24/12/02 16:00:40 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
24/12/02 16:00:40 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
24/12/02 16:00:40 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
24/12/02 16:00:40 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
24/12/02 16:00:40 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.


+-----------------+-------------------------------------------+----------------------+-----+
|code             |product_name                               |brands                |count|
+-----------------+-------------------------------------------+----------------------+-----+
|3.250392171018E12|Poulet curry                               |Monique Ranou         |2    |
|3.254566480876E12|Queues de crevettes cocktail cuites        |Auchan                |2    |
|3.560070437917E12|Lait demi-écrémé                           |Carrefour Bio         |2    |
|3.25039342917E12 |Moutarde                                   |Bouton d'Or           |2    |
|3.560070906833E12|Tarti'Bon Ail & fines herbes               |Carrefour             |2    |
|3.560071193751E12|Pomme banane Dessert de fruits             |Carrefour             |2    |
|2.030500401531E12|Croissant                                  |Auchan                |2    |
|3.25039174617E12 |Mais doux en grains                        |Saint E

Number of rows after removing duplicates: 3511425
Execution completed in 13.38 seconds


#### Suppression des doublons

In [7]:
from pyspark.sql.functions import regexp_extract

df_parquet = df_parquet.withColumn(
    "quantity_numeric",
    regexp_extract(col("quantity"), r"(\d+)", 1).cast("double")
)

numeric_columns = [
    field.name for field in df_parquet.schema.fields
    if str(field.dataType) in ["IntegerType", "DoubleType", "FloatType"]
]
print(f"Numeric columns detected: {numeric_columns}")

if not numeric_columns:
    print("No numeric columns found. Please check your data.")
else:
    # Boucle sur les colonnes numériques pour détecter les outliers
    for column in numeric_columns:
        try:
            quantiles = df_parquet.approxQuantile(column, [0.25, 0.75], 0.1)
            if len(quantiles) < 2:
                print(f"Column '{column}' has insufficient data. Skipping...")
                continue

            q1, q3 = quantiles
            iqr = q3 - q1
            lower_bound = q1 - 1.5 * iqr
            upper_bound = q3 + 1.5 * iqr

            print(f"Column: {column}")
            print(f"Q1: {q1}, Q3: {q3}, IQR: {iqr}")
            print(f"Lower Bound: {lower_bound}, Upper Bound: {upper_bound}")

            outliers = df_parquet.filter((col(column) < lower_bound) | (col(column) > upper_bound))
            print(f"Outliers detected in '{column}': {outliers.count()}")

        except Exception as e:
            print(f"Error processing column '{column}': {e}")


Numeric columns detected: []
No numeric columns found. Please check your data.


### Normalisation des colonnes

In [8]:
df_parquet.describe()


DataFrame[summary: string, code: string, url: string, creator: string, created_t: string, last_modified_t: string, last_modified_by: string, last_updated_t: string, product_name: string, abbreviated_product_name: string, generic_name: string, quantity: string, packaging: string, packaging_tags: string, packaging_en: string, packaging_text: string, brands: string, brands_tags: string, categories: string, categories_tags: string, categories_en: string, origins: string, origins_tags: string, origins_en: string, manufacturing_places: string, manufacturing_places_tags: string, labels: string, labels_tags: string, labels_en: string, emb_codes: string, emb_codes_tags: string, first_packaging_code_geo: string, cities: string, cities_tags: string, purchase_places: string, stores: string, countries: string, countries_tags: string, countries_en: string, ingredients_text: string, ingredients_tags: string, ingredients_analysis_tags: string, allergens: string, allergens_en: string, traces: string, t

####

In [24]:
selected_column = [
    'code',
    'product_name',
    'brands',
    'categories',
    "main_category",
    'quantity',
    'packaging',
    'countries',
    'ingredients_text',
    'allergens',
    'serving_size',
    'energy-kcal_100g',
    'fat_100g',
    'saturated-fat_100g',
    "proteins_100g",
    'sugars_100g',
    'salt_100g',
    'nutriscore_score',
    'nutriscore_grade',
    "food_groups_en",
]

df_transformed = df_parquet.select(selected_column)
df_transformed.show(5, truncate=False)

+-----------------+-----------------------------------------------+------------------------------+-------------------------------------------------------------+----------------+---------+------------------------+---------+----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [51]:
# convertir les donnés string en integer
column_to_convert = ["quantity", "nutriscore_score", "energy-kcal_100g",
                     "fat_100g", "saturated-fat_100g", "proteins_100g", "sugars_100g", "salt_100g"]
# apply the conversion
for column in column_to_convert:
    df_transformed = df_transformed.withColumn(column, col(column).cast("double"))
df_transformed.printSchema()

root
 |-- code: string (nullable = true)
 |-- product_name: string (nullable = true)
 |-- brands: string (nullable = true)
 |-- categories: string (nullable = true)
 |-- main_category: string (nullable = true)
 |-- quantity: double (nullable = true)
 |-- packaging: string (nullable = true)
 |-- countries: string (nullable = true)
 |-- ingredients_text: string (nullable = true)
 |-- allergens: string (nullable = true)
 |-- serving_size: string (nullable = true)
 |-- energy-kcal_100g: double (nullable = true)
 |-- fat_100g: double (nullable = true)
 |-- saturated-fat_100g: double (nullable = true)
 |-- proteins_100g: double (nullable = true)
 |-- sugars_100g: double (nullable = true)
 |-- salt_100g: double (nullable = true)
 |-- nutriscore_score: double (nullable = true)
 |-- nutriscore_grade: string (nullable = true)
 |-- food_groups_en: string (nullable = true)



In [52]:
df_transformed.show(5, truncate=False)
# convert code in string
df_transformed = df_transformed.withColumn("code", col("code").cast("string"))

+-----------------+-----------------------------------------------+------------------------------+-------------------------------------------------------------+----------------+--------+------------------------+---------+-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [53]:
from pyspark.sql.functions import col, sum

# Calculer le nombre de valeurs nulles par colonne
null_counts = df_transformed.select([sum(col(c).isNull().cast("int")).alias(c) for c in df_transformed.columns])

# Afficher les résultats
null_counts.show()


+----+------------+-------+----------+-------------+--------+---------+---------+----------------+---------+------------+----------------+--------+------------------+-------------+-----------+---------+----------------+----------------+--------------+
|code|product_name| brands|categories|main_category|quantity|packaging|countries|ingredients_text|allergens|serving_size|energy-kcal_100g|fat_100g|saturated-fat_100g|proteins_100g|sugars_100g|salt_100g|nutriscore_score|nutriscore_grade|food_groups_en|
+----+------------+-------+----------+-------------+--------+---------+---------+----------------+---------+------------+----------------+--------+------------------+-------------+-----------+---------+----------------+----------------+--------------+
|   0|      192029|1474270|   1986299|      1986311| 3489285|  3133427|    24467|         2506091|  3209532|     2616448|          921149|  903806|           1013453|       898286|     978704|  1246857|         2342750|           22934|       2

In [54]:
#drop missing values in columns
df_no_duplicates = df_transformed.dropDuplicates()


In [55]:
from pyspark.sql.functions import col, sum

# Calculer le nombre de valeurs nulles par colonne
null_counts = df_no_duplicates.select([sum(col(c).isNull().cast("int")).alias(c) for c in df_no_duplicates.columns])

# Afficher les résultats
null_counts.show()

+----+------------+-------+----------+-------------+--------+---------+---------+----------------+---------+------------+----------------+--------+------------------+-------------+-----------+---------+----------------+----------------+--------------+
|code|product_name| brands|categories|main_category|quantity|packaging|countries|ingredients_text|allergens|serving_size|energy-kcal_100g|fat_100g|saturated-fat_100g|proteins_100g|sugars_100g|salt_100g|nutriscore_score|nutriscore_grade|food_groups_en|
+----+------------+-------+----------+-------------+--------+---------+---------+----------------+---------+------------+----------------+--------+------------------+-------------+-----------+---------+----------------+----------------+--------------+
|   0|      191972|1474242|   1985858|      1985870| 3487570|  3132270|    24465|         2505268|  3208240|     2615071|          920827|  903501|           1013148|       897985|     978400|  1246432|         2342080|           22932|       2